# common

In [211]:
import json

def load(output_path, target_path):
    with open(output_path, "r", encoding="utf-8") as file:
        output = json.load(file)
    with open(target_path, "r", encoding="utf-8") as file:
        target = json.load(file)
    print(len(output))
    for o in output[:5]:
        print(o)
    print(len(target))
    for t in target[:5]:
        print(t)
    return output, target

In [212]:
def save_diff(output, target, diff_path):
    count = 0
    with open(diff_path, "w", encoding="utf-8") as file:
        for i in range(len(output)):
            o = output[i]
            t = target[i]
            if o == t:
                continue
            count += 1
            o_set = set()
            t_set = set()
            o_map = {}
            t_map = {}
            curr_len = 0
            for j in range(len(o)):
                start = curr_len
                curr_len += len(o[j])
                end = curr_len
                o_set.add((start, end))
                o_map[(start, end)] = j
            curr_len = 0
            for j in range(len(t)):
                start = curr_len
                curr_len += len(t[j])
                end = curr_len
                t_set.add((start, end))
                t_map[(start, end)] = j
            intersection = o_set.intersection(t_set)
            o_diff_set = o_set - intersection
            t_diff_set = t_set - intersection
            o_diff_index = sorted([o_map[intv] for intv in o_diff_set])
            t_diff_index = sorted([t_map[intv] for intv in t_diff_set])
            o_diff = [o[j] for j in o_diff_index]
            t_diff = [t[j] for j in t_diff_index]
            file.write(str(i) + "\n")
            file.write("".join(t) + "\n")
            file.write(str(o_diff) + "\n")
            file.write(str(t_diff) + "\n")
            file.write("\n")
    print(count)

In [213]:
import numpy as np


def char_level_eval(output, target):
    # 0: S, 1: B, 2:I, 3:E
    matrix = np.zeros((5, 5))
    for i in range(len(output)):
        o = output[i]
        t = target[i]
        o_chars = []
        t_chars = []
        for o_word in o:
            if len(o_word) == 1:
                o_chars.append(0)
            else:
                o_chars.append(1)
                for _ in range(1, len(o_word) - 1):
                    o_chars.append(2)
                o_chars.append(3)
        for t_word in t:
            if len(t_word) == 1:
                t_chars.append(0)
            else:
                t_chars.append(1)
                for _ in range(1, len(t_word) - 1):
                    t_chars.append(2)
                t_chars.append(3)
        for j in range(min(len(o_chars), len(t_chars))):
            matrix[t_chars[j], o_chars[j]] += 1
    for i in range(4):
        matrix[i, 4] = matrix[i, i] / matrix[i, :4].sum()
        matrix[4, i] = matrix[i, i] / matrix[:4, i].sum()
    np.set_printoptions(precision=4, suppress=True)
    precision = matrix[4, :4].mean()
    recall = matrix[:4, 4].mean()
    f1 = 2 * precision * recall / (precision + recall)
    print("Character-level evaluation:")
    print("Precision: %.4f" % precision)
    print("Recall: %.4f" % recall)
    print("F1: %.4f" % f1)
    print(matrix)

In [214]:
def word_level_eval(output, target):
    num_correct = 0
    num_predicted = 0
    num_truth = 0
    for i in range(len(output)):
        o = output[i]
        t = target[i]
        o_intervals = set()
        t_intervals = set()
        curr_len = 0
        for j in range(len(o)):
            start = curr_len
            curr_len += len(o[j])
            end = curr_len
            o_intervals.add((start, end))
        curr_len = 0
        for j in range(len(t)):
            start = curr_len
            curr_len += len(t[j])
            end = curr_len
            t_intervals.add((start, end))
        num_correct += len(o_intervals.intersection(t_intervals))
        num_predicted += len(o_intervals)
        num_truth += len(t_intervals)
    precision = num_correct / num_predicted
    recall = num_correct / num_truth
    f1 = 2 * precision * recall / (precision + recall)
    print("Word-level evaluation:")
    print("Precision: %.4f" % precision)
    print("Recall: %.4f" % recall)
    print("F1: %.4f" % f1)

# msr

In [215]:
msr_output_path = "D:\\Local\\Workspace\\CSE256\\AdaSeq\\playground\\outputs\\msr_output.json"
msr_target_path = "D:\\Local\\Workspace\\CSE256\\AdaSeq\\playground\\outputs\\msr_target.json"
msr_output, msr_target = load(msr_output_path, msr_target_path)

3985
['扬帆', '远东', '做', '与', '中国', '合作', '的', '先行']
['希腊', '的', '经济', '结构', '较', '特殊', '。']
['海运业', '雄踞', '全球', '之', '首', '，', '按', '吨位', '计', '占', '世界', '总数', '的', '１７％', '。']
['另外', '旅游', '、', '侨汇', '也', '是', '经济', '收入', '的', '重要', '组成部分', '，', '制造业', '规模', '相对', '较', '小', '。']
['多', '年', '来', '，', '中', '希', '贸易', '始终', '处于', '较', '低', '的', '水平', '，', '希腊', '几乎', '没有', '在', '中国', '投资', '。']
3985
['扬帆', '远东', '做', '与', '中国', '合作', '的', '先行']
['希腊', '的', '经济', '结构', '较', '特殊', '。']
['海运', '业', '雄踞', '全球', '之', '首', '，', '按', '吨位', '计', '占', '世界', '总数', '的', '１７％', '。']
['另外', '旅游', '、', '侨汇', '也是', '经济', '收入', '的', '重要', '组成部分', '，', '制造业', '规模', '相对', '较小', '。']
['多年来', '，', '中', '希', '贸易', '始终', '处于', '较低', '的', '水平', '，', '希腊', '几乎', '没有', '在', '中国', '投资', '。']


In [216]:
msr_diff_path = "D:\\Local\\Workspace\\CSE256\\AdaSeq\\playground\\diffs\\diff_msr.txt"
save_diff(msr_output, msr_target, msr_diff_path)

3148


In [217]:
char_level_eval(msr_output, msr_target)
word_level_eval(msr_output, msr_target)

Character-level evaluation:
Precision: 0.8765
Recall: 0.8276
F1: 0.8514
[[43835.      1600.       388.      2261.         0.9116]
 [ 2946.     55318.       477.        40.         0.9411]
 [  483.      4136.      9878.      4204.         0.5282]
 [ 2961.        93.      1085.     54642.         0.9296]
 [    0.8728     0.9047     0.8351     0.8936     0.    ]]
Word-level evaluation:
Precision: 0.8516
Recall: 0.8876
F1: 0.8692


# pku

In [218]:
pku_output_path = "D:\\Local\\Workspace\\CSE256\\AdaSeq\\playground\\outputs\\pku_output.json"
pku_target_path = "D:\\Local\\Workspace\\CSE256\\AdaSeq\\playground\\outputs\\pku_target.json"
pku_output, pku_target = load(pku_output_path, pku_target_path)

1938
['共同', '创造', '美好', '的', '新', '世纪', '——', '二○○一年', '新年', '贺词']
['（', '二○○○年', '十二月', '三十一日', '）', '（', '附', '图片', '1', '张', '）']
['女士', '们', '，', '先生', '们', '，', '同志', '们', '，', '朋友', '们', '：']
['2001年', '新年', '钟声', '即将', '敲响', '。', '人类', '社会', '前进', '的', '航船', '就要', '驶入', '21', '世纪', '的', '新', '航程', '。', '中国', '人民', '进入', '了', '向', '现代化', '建设', '第三', '步', '战略', '目标', '迈进', '的', '新', '征程', '。']
['在', '这个', '激动人心', '的', '时刻', '，', '我', '很', '高兴', '通过', '中国', '国际', '广播', '电台', '、', '中央', '人民', '广播', '电台', '和', '中央', '电视台', '，', '向', '全国', '各族', '人民', '，', '向', '香港', '特别', '行政区', '同胞', '、', '澳门', '特别', '行政区', '同胞', '和', '台湾', '同胞', '、', '海外', '侨胞', '，', '向', '世界', '各国', '的', '朋友', '们', '，', '致以', '新', '世纪', '第一', '个', '新年', '的', '祝贺', '！']
1938
['共同', '创造', '美好', '的', '新', '世纪', '——', '二○○一年', '新年', '贺词']
['（', '二○○○年', '十二月', '三十一日', '）', '（', '附', '图片', '1', '张', '）']
['女士', '们', '，', '先生', '们', '，', '同志', '们', '，', '朋友', '们', '：']
['2001年', '新年', '钟声', '即将', '敲响', '。', '人类', '社会', 

In [219]:
pku_diff_path = "D:\\Local\\Workspace\\CSE256\\AdaSeq\\playground\\diffs\\diff_pku.txt"
save_diff(pku_output, pku_target, pku_diff_path)

969


In [220]:
char_level_eval(pku_output, pku_target)
word_level_eval(pku_output, pku_target)

Character-level evaluation:
Precision: 0.9564
Recall: 0.9707
F1: 0.9635
[[44633.       752.       281.       844.         0.9596]
 [  498.     54501.       518.        30.         0.9812]
 [   65.       198.     10731.       164.         0.9617]
 [  459.        35.       605.     54448.         0.9802]
 [    0.9776     0.9822     0.8843     0.9813     0.    ]]
Word-level evaluation:
Precision: 0.9727
Recall: 0.9639
F1: 0.9683


# nlpcc

In [221]:
nlpcc_output_path = "D:\\Local\\Workspace\\CSE256\\AdaSeq\\playground\\outputs\\nlpcc_output.json"
nlpcc_target_path = "D:\\Local\\Workspace\\CSE256\\AdaSeq\\playground\\outputs\\nlpcc_target.json"
nlpcc_output, nlpcc_target = load(nlpcc_output_path, nlpcc_target_path)

2052
['【', '神秘', '中国', '财团', '本周', '或', '买', '下', 'AC', '米兰', '，', '代价', '超', '10亿', '欧元', '】']
['【', '缩量', '回调', '，', '3000', '点', '还要', '徘徊', '多久', '？', '】']
['【', '美国', '再', '启动', '337', '调查', '：', '联想', '中兴', '或', '遭', '重拳', '】']
['【', '联盛', '破产', '冲击波', '：', '山西', '宏特', '资金链', '断裂', '，', '债委会', '难', '救', '煤焦油', '龙头', '】']
['【', '自由', '教师', '：', '年收入', '过', '百万', '的', '新', '工作', '？', '】']
2052
['【', '神秘', '中国', '财团', '本周', '或', '买下', 'AC米兰', '，', '代价', '超', '10亿', '欧元', '】']
['【', '缩量', '回调', '，', '3000', '点', '还要', '徘徊', '多久', '？', '】']
['【', '美国', '再', '启动', '337', '调查', '：', '联想', '中兴', '或', '遭', '重拳', '】']
['【', '联盛', '破产', '冲击波', '：', '山西', '宏特', '资金链', '断裂', '，', '债委会', '难救', '煤焦油', '龙头', '】']
['【', '自由', '教师', '：', '年收入', '过', '百万', '的', '新', '工作', '？', '】']


In [222]:
nlpcc_diff_path = "D:\\Local\\Workspace\\CSE256\\AdaSeq\\playground\\diffs\\diff_nlpcc.txt"
save_diff(nlpcc_output, nlpcc_target, nlpcc_diff_path)

1346


In [223]:
char_level_eval(nlpcc_output, nlpcc_target)
word_level_eval(nlpcc_output, nlpcc_target)

Character-level evaluation:
Precision: 0.9245
Recall: 0.9186
F1: 0.9215
[[19310.       576.        64.       578.         0.9407]
 [ 1292.     21590.       264.        23.         0.9318]
 [  111.       408.      5527.       330.         0.8668]
 [ 1214.        23.       266.     21666.         0.9351]
 [    0.8806     0.9554     0.903      0.9588     0.    ]]
Word-level evaluation:
Precision: 0.9061
Recall: 0.9232
F1: 0.9146


# medical

In [224]:
medical_output_path = "D:\\Local\\Workspace\\CSE256\\AdaSeq\\playground\\outputs\\medical_output.json"
medical_target_path = "D:\\Local\\Workspace\\CSE256\\AdaSeq\\playground\\outputs\\medical_target.json"
medical_output, medical_target = load(medical_output_path, medical_target_path)

907
['她', '现在', '85', '岁', '了', '，', '生活', '能', '自理', '，', '自己', '能', '做饭', '，', '能', '骑', '电瓶车', '，', '今天', '有', '剧烈', '咳嗽', '，', '有', '痰']
['5', '年', '前', '感冒', '咳嗽', '有', '半', '年', '时间', '服', '止咳', '糖浆', '等', '！']
['半', '年', '后', '腿', '肿', '了', '血压', '高', '140', '！']
['住院', '拍', '肺部', 'CT', '显示', '肺栓塞', '！']
['现在', '走路', '快', '了', '就', '气喘', '！']
907
['她', '现在', '85', '岁', '了', '，', '生活', '能', '自理', '，', '自己', '能', '做饭', '，', '能', '骑', '电瓶车', '，', '今天', '有', '剧烈', '咳嗽', '，', '有', '痰']
['5', '年', '前', '感冒', '咳嗽', '有', '半年', '时间', '服', '止咳', '糖浆', '等', '！']
['半年', '后', '腿', '肿', '了', '血压', '高', '140', '！']
['住院', '拍', '肺部', 'CT', '显示', '肺栓塞', '！']
['现在', '走路', '快', '了', '就', '气喘', '！']


In [225]:
medical_diff_path = "D:\\Local\\Workspace\\CSE256\\AdaSeq\\playground\\diffs\\diff_medical.txt"
save_diff(medical_output, medical_target, medical_diff_path)

612


In [226]:
char_level_eval(medical_output, medical_target)
word_level_eval(medical_output, medical_target)

Character-level evaluation:
Precision: 0.8174
Recall: 0.7910
F1: 0.8040
[[5604.      326.       16.      407.        0.8821]
 [ 804.     5800.      119.       20.        0.8602]
 [ 145.      221.      741.      195.        0.5691]
 [ 773.       25.      195.     5750.        0.8527]
 [   0.7649    0.9102    0.6919    0.9024    0.    ]]
Word-level evaluation:
Precision: 0.8081
Recall: 0.8453
F1: 0.8263
